In [32]:
dimension = 3
perimeter = 50
sensitivity = 1
jsonfilename = "matrices3x3alotofinstances.json"

In [19]:
# UNINTERESTING TOOLS
from random import randint, seed


def shortestVector(matrix): 
    """
    :returns: 
        - n - norm of the shortest vector
        - v - the shortest vector (vector)
        - i - index of the SV
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per) -> matrix:
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]


def into_dict(B, G, x, solutions) -> dict:
    result = {}
    result["B"] = B
    result["G"] = G
    result["lincomb_LLL"] = x
    result["lincomb_cube"] = solutions
    result["sv_LLL"] = x*B
    result["sv_cube"] = vector(solutions)*B
    result["lincomb_diff"] = x - vector(solutions)
    result["Diff impact"] = matrix_multiplication_detailed(G, x - vector(solutions))[0]
#     result["Diff impact sum"] = matrix_multiplication_detailed(B, x - vector(solutions))[1]
#     result["LLL.norm"] = vector(x*B).norm().n(digits=5)
#     result["cube.norm"] = (vector(solutions)*B).norm().n(digits=5)
    return result

In [18]:
### batshit cool math
def innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) -> dict:
    solutions = find_real_minimum(G, i, x[i])
    for j in range(dimension - 1):  # zbytecny forcyklus???
        difference = x[j] - solutions[j]
        if abs(difference) > sensitivity: 
            if all([sol == 0 for sol in solutions]):# FIXME fuj
                break  
            return into_dict(B, G, x, solutions)
        
def find_real_minimum(G, i, x) -> vector:
    matrixA = matrix(G.nrows() - 1, G.nrows() - 1, 0)
    matrixB = matrix(G.nrows() - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(G.nrows()):
        if row != i:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != i]
            matrixB[b] = sum([x * G[row,j] for j in range(len(G[row])) if j == i])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*matrixB)).list()
    result.insert(i,x)
    return vector(result).n(digits=5) 


def matrix_multiplication_detailed(A, vector) -> (matrix, float):
    """
    :returns: 
        - A - matrix A multiplied in rows and cols by vector
        - x - sum of all elements in the returned matrix
    """
    A = matrix_to_list(A)
    n = len(A)
    diff = vector_to_list(diff)
    for i in range(n):
        for j in range(n):
            A[i][j] = round(B[i][j]*  diff[i] * diff[j], 2)
    return matrix(A), round(sum(sum(A, [])), 2)

In [17]:
### output formatting
import json

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            else:
            # FIXME here I assume that output_data consits only of matrices and one-line entries (vectors, free modules)
            # it IS going to break if I try to save integers/floats
                dic[key] = vector_to_list(value)
    return output_data

def into_json(output_data):
    out_file = open(jsonfilename, "w+") 
    json.dump(output_data, out_file) 
    out_file.close() 

def from_json(input_data):
    out_file = open(jsonfilename)
    return json.load(out_file) # close it maybe?

def print_dict(dictionary) -> None:
    for pair in dictionary.items():
        print(pair[1], ": ", pair[0])
    print()
    
    

In [30]:
# MAIN 
def generate_new_examples(iterations) -> None:
    output_data = []
    for _ in range(iterations):
        B = randomMatrix(dimension, perimeter)
        data = abnormality_check(B)
        if data != None: #FIXME terrible, can I get rid of these ifs somehow?
            output_data.append(abnormality_check(B))
    for dictionary in output_data:
        print_dict(dictionary)
    into_json(format_data(output_data))


def abnormality_check(B) -> dict:
    B_red = B.LLL()
    v_min = shortestVector(B_red)[1]
    x = B.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
    G = gram_matrix(B)
    for i in range(G.nrows()):
        data = innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) # terrible
        if data != None:
            return data

In [33]:
# generate_new_examples(100000)

[ 12  12 -10]
[-39 -28  32]
[ 19 -36  -4] :  B
[  388 -1124  -164]
[-1124  3329   139]
[ -164   139  1673] :  G
(15, 5, 1) :  lincomb_LLL
(13.783, 4.6118, 1.0000) :  lincomb_cube
(4, 4, 6) :  sv_LLL
(4.5315, 0.26147, 5.7512) :  sv_cube
(1.2173, 0.38818, 0.00000) :  lincomb_diff
[ 574.93 -531.11    -0.0]
[-531.11  501.62     0.0]
[   -0.0     0.0     0.0] :  Diff impact

[ 33 -28 -26]
[ 12  -7   0]
[ -5   3  21] :  B
[2549  592 -795]
[ 592  193  -81]
[-795  -81  475] :  G
(1, -3, 1) :  lincomb_LLL
(0.74550, -1.8670, 1.0000) :  lincomb_cube
(-8, -4, -5) :  sv_LLL
(-2.8028, -4.8048, 1.6170) :  sv_cube
(0.25450, -1.1330, 0.00000) :  lincomb_diff
[ 165.1 -170.7   -0.0]
[-170.7 247.74    0.0]
[  -0.0    0.0    0.0] :  Diff impact

[  4  26  -8]
[ 24  -1 -38]
[ 47 -29 -38] :  B
[ 756  374 -262]
[ 374 2021 2601]
[-262 2601 4494] :  G
(1, 0, 0) :  lincomb_LLL
(1.0000, -1.0194, 0.64832) :  lincomb_cube
(4, 26, -8) :  sv_LLL
(10.005, 8.2181, 6.1024) :  sv_cube
(0.00000, 1.0194, -0.64832) :  linco